In [2]:
import kfp
import kfp.dsl as dsl
from kfp.v2.dsl import (
    component,
    Input,
    Output,
    Dataset,
    Metrics,
    pipeline,
    Artifact,
)

In [ ]:
def drop_cols(df, cols_to_drop):
    df = df.drop(cols_to_drop, axis=1)
    return df

In [4]:
@component(packages_to_install=['pandas'])
def drop_cols(df_artifact: Input[Artifact], cols_to_drop_string: str, output_artifact: Output[Artifact])->None:
    import pandas as pd 
    import json 
    cols_to_drop = json.loads(cols_to_drop_string)
    df = pd.read_csv(df_artifact.path)
    df = df.drop(cols_to_drop, axis=1)
    df.to_csv(output_artifact.path)

In [3]:
@component(packages_to_install=['pandas', 'sklearn'])
def preprocessing(df_artifact: Input[Artifact], x_artifact: Output[Artifact], y_artifact: Output[Artifact])->None:
    import pandas as pd 
    from sklearn.preprocessing import LabelEncoder, OneHotEncoder, RobustScaler
    le = LabelEncoder()
    ohe = OneHotEncoder()
    scaler = RobustScaler()

    df = pd.read_csv(df_artifact.path)

    def drop_cols(df, cols_to_drop):
        df = df.drop(cols_to_drop, axis=1)
        return df

    def encode_sex(df):
        df['Male'] = le.fit_transform(df.Sex)
        df = df.drop(['Sex'], axis=1)
        return df

    def imputation_by_group(df, column_to_impute, columns_to_group_by):
        """Impute the column_to_impute by taking the median group on columns_to_group_by"""
        df[column_to_impute] = df.groupby(columns_to_group_by)[column_to_impute].apply(lambda x: x.fillna(x.median()))
        return df 

    def fill_categorical_most_common(df, column_to_fill):
        """Fill nulls in categorical column_to_fill with the most common variable"""
        most_common = df[column_to_fill].mode()[0]
        df[column_to_fill].fillna(most_common)
        return df

    def one_hot_encoding(df, col_to_encode):
        """One hot encodes categorical variable col_to_encode and drops the least common column"""
        result = ohe.fit_transform(df[col_to_encode].values.reshape(-1,1)).toarray()

        feature_names = ohe.get_feature_names().tolist()
        df[feature_names] = pd.DataFrame(result, index=df.index)

        least_common = df[col_to_encode].value_counts().index[-1]
        col_to_drop = 'x0_'+least_common
        df = df.drop([col_to_encode, col_to_drop], axis=1)
        return df 

    def robust_scaling(df, col_to_scale):
        df[col_to_scale] = scaler.fit_transform(train_df[col_to_scale].values.reshape(-1,1))
        return df

    def binary_binning(df, col_to_bin):
        """Takes col_to_bin and creates binary count for 0 or 1+"""
        correction = lambda x: 1 if x != 0 else 0
        df[col_to_bin] = df[col_to_bin].apply(correction)
        return df

    non_informative_cols = ['PassengerId', 'Name', 'Ticket']
    df = drop_cols(df, non_informative_cols)

    mostly_null_cols = ['Cabin']
    df = drop_cols(df, mostly_null_cols)

    df = encode_sex(df)

    df = imputation_by_group(df, 'Age', ['Pclass', 'Male'])
    df = imputation_by_group(df, 'Fare', 'Pclass')

    df = fill_categorical_most_common(df, 'Embarked')

    df = one_hot_encoding(df, 'Embarked')

    df = robust_scaling(df, 'Age')
    df = robust_scaling(df, 'Fare')

    df = binary_binning(df, 'SibSp')
    df = binary_binning(df, 'Parch')

    y = df.Survived
    x = df.drop('Survived', axis=1)

    x.to_csv(x_artifact.path)
    y.to_csv(y_artifact.path)

In [4]:
@component(packages_to_install=['pandas', 'sklearn'])
def train_model(x_artifact: Input[Artifact], y_artifact: Input[Artifact], model_artifact: Output[Artifact])->None:
    import pandas as pd 
    import pickle
    from sklearn.linear_model import LogisticRegression
    logreg = LogisticRegression()

    x = pd.read_csv(x_artifact.path)
    y = pd.read_csv(y_artifact.path)

    logreg.fit(x,y)

    pickle.dump(logreg, open(model_artifact.path, 'wb'))

In [ ]:
@component(packages_to_install=['pandas', 'sklearn'])
def predict(model_artifact: Input[Artifact], x_test_artifact: Input[Artifact], y_pred_artifact: Output[Artifact])->None:
    import pandas as pd 
    import pickle 

    model = pickle.load(open(model_artifact.path, 'rb'))

    x_test = pd.read_csv(x_test_artifact.path)
    y_pred = model.predict(x_test)
    y_pred.to_csv(y_pred_artifact.path)


In [ ]:
@component(packages_to_install=['pandas', 'sklearn'])
def evaluate(y_test_artifact: Input[Artifact], y_pred_artifact: Input[Artifact])->float:
    import pandas as pd
    import sklearn 
    from sklearn.metrics import accuracy_score

    y_test = pd.read_csv(y_test_artifact.path)
    y_pred = pd.read_csv(y_pred_artifact.path)

    accuracy = accuracy_score(y_test, y_pred)
    return accuracy


In [ ]:
@component(packages_to_install = ['google-cloud-storage']
)
def download_gcs_file(
project : str,
bucket : str,
filepath : str,
output_file : Output[Artifact]
) -> None:
'''Downloads a file from GCS to a KFP Artifact component'''
from google.cloud import storage

client = storage.Client(project = project)
bucket = client.get_bucket(bucket)
blob = bucket.blob(filepath)

blob.download_to_filename(output_file.path)

In [ ]:
@pipeline('titanic_pipeline')
def create_pipleine(cols_to_drop_string: str):
    download_gcs_file_op = download_gcs_file(project_string, bukcet_string, filepath_string)

    drop_cols_op = drop_cols(download_gcs_file_op.output, cols_to_drop_string)
    

In [ ]:
@pipeline(name='titanic_pipleine', description='dummy pipeline on titanic dataset')
def build_pipeline():
    preprocessing_op = preprocessing(df)
    train_model_op = train_model(preprocessing_op.output)
    predict_op = predict(train_model_op.output, x_test)
    evaluate_op = evaluate(y_test, predict_op.output)